In [ ]:
#import timeit
%load_ext autotime
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

In [ ]:

#start_time = timeit.default_timer()
# lstm autoencoder recreate sequence


# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))
# define model
model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(10, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
print('1')
# fit model
model.fit(sequence, sequence, epochs=200, verbose=0)
print(2)
#plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(3)
print(yhat[0,:,0])
#elapsed = timeit.default_timer() - start_time
#elapsed

In [ ]:
import numpy as np
a=np.genfromtxt('data_for_clustering.csv', delimiter=',')
a=a[:-1]

In [ ]:
# lstm autoencoder recreate sequence
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
# define input sequence
sequence = a[0:250]
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))
# define model
model = Sequential()
model.add(LSTM((10), activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM((10), activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(sequence, sequence, epochs=30, verbose=0)
#plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard
import numpy as np 
from tensorflow import set_random_seed
import os

In [3]:
import numpy as np
a=np.genfromtxt('data_for_clustering.csv', delimiter=',')
a=a[:-1]
print(a[1])
m=np.mean(a)
a=a-m
print(a[1])
print(m)
print(a.shape)
x=a[:250*910]
print(x[0],x[1])
x=x.reshape(-1,250)
print(x[0][0],x[0][1])


39963.59
9067.127637781465
30896.46236221853
(228260,)
9080.487637781465 9067.127637781465
9080.487637781465 9067.127637781465


In [ ]:
def seedy(s):
    np.random.seed(s)
    set_random_seed(s)
class AutoEncoder:
    def __init__(self, encoding_dim=3):
        self.encoding_dim = encoding_dim
        #r = lambda: np.random.randint(1, 3)
        #self.x = np.array([[r(),r(),r()] for _ in range(1000)])
        self.x=x
        print(self.x.shape)

    def _encoder(self):
        inputs = Input(shape=(self.x[0].shape))
        encoded = Dense(self.encoding_dim, activation='tanh')(inputs)
        model = Model(inputs, encoded)
        self.encoder = model
        return model

    def _decoder(self):
        inputs = Input(shape=(self.encoding_dim,))
        decoded = Dense(250)(inputs)
        model = Model(inputs, decoded)
        self.decoder = model
        return model

    def encoder_decoder(self):
        ec = self._encoder()
        dc = self._decoder()
        
        inputs = Input(shape=self.x[0].shape)
        ec_out = ec(inputs)
        dc_out = dc(ec_out)
        model = Model(inputs, dc_out)
        
        self.model = model
        return model

    def fit(self, batch_size=10, epochs=300):
        self.model.compile(optimizer='sgd', loss='mse')
        log_dir = './log/'
        tbCallBack = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
        self.model.fit(self.x, self.x,
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[tbCallBack])

    def save(self):
        if not os.path.exists(r'./weights'):
            os.mkdir(r'./weights')
        else:
            self.encoder.save(r'./weights/encoder_weights.h5')
            self.decoder.save(r'./weights/decoder_weights.h5')
            self.model.save(r'./weights/ae_weights.h5')

In [ ]:
if __name__ == '__main__':
    seedy(2)
    ae = AutoEncoder(encoding_dim=10)
    ae.encoder_decoder()
    ae.fit(batch_size=910, epochs=200)
    ae.save()

In [ ]:
from keras.models import load_model
import numpy as np

encoder = load_model(r'./weights/encoder_weights.h5')
decoder = load_model(r'./weights/decoder_weights.h5')

inputs = np.array([x[0]])
X = encoder.predict(inputs)
y = decoder.predict(X)

print('Input: {}'.format(inputs))
print('Encoded: {}'.format(X))
print('Decoded: {}'.format(y))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
plt.plot(range(250),inputs.reshape(250),label=1)
plt.plot(range(250),y.reshape(250),label=2)

In [ ]:
inputs.shape

In [1]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:

# this is the size of our encoded representations
encoding_dim = 10 # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
 
# this is our input placeholder
input_img = Input(shape=(250,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(250, activation='sigmoid')(encoded)
# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))
# configure our model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()
# normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [4]:
print(x.shape)
autoencoder.fit(x,x,
epochs=50,
batch_size=256,
shuffle=True)

#validation_data=(x, x_test))
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x)
decoded_imgs = decoder.predict(encoded_imgs)

(910, 250)
Instructions for updating:
Use tf.cast instead.
Epoch 1/50


InternalError: Blas GEMM launch failed : a.shape=(256, 250), b.shape=(250, 10), m=256, n=10, k=250
	 [[{{node dense_1/MatMul}}]]
	 [[{{node loss/mul}}]]

In [ ]:
%matplotlib notebook
n = 20 # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
# display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
 
 
 
# display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
print(encoded_imgs.shape)

In [ ]:
encoded_imgs[0]